In [4]:
from pyspark.sql.functions import col, lit, avg
import datetime


In [5]:
new_log_data = sc.textFile("hdfs://localhost:54310/data1/usr_log_data.csv").map(lambda x:x.split(","))


In [6]:
# creating dataframe from csv file
# csv filtered to extract columns and values
# in schema index 0 is to collect first row, which is headers
new_log_dataframe = sqlContext.createDataFrame(data=new_log_data.filter(lambda x:x[0]!='user_name'),
                             schema=new_log_data.filter(lambda x:x[0]=='user_name').collect()[0])

In [7]:
# creating new column list to store extracted seconds and username(as primary key) from dataframe 
new_column_list = []
for row in new_log_dataframe.collect():
    # converting string type of datetime to datetime
    date_time = datetime.datetime.strptime(row['working_hours'], "%Y-%m-%d %H:%M:%S")
    converted_seconds = date_time.hour * 3600 + date_time.minute *60 + date_time.second
    new_column_list.append((row['user_name'], converted_seconds))

In [8]:
# creating new column dataframe to join with original dataframe( new_log_dataframe)
new_column = sqlContext.createDataFrame(new_column_list, ('username',"seconds"))

In [10]:
# joining new column to new_log_dataframe using username as primary key
joined_dataframe = (new_log_dataframe
    .join(new_column, col("user_name")==col("username"),"leftouter")
    .drop("username")
   )

In [11]:
# Generating average seconds
average_object = joined_dataframe.agg(avg(col("seconds")))

In [12]:
# unpacking average value from average_object|
average_value = average_object.collect()[0][0]

In [13]:
# filtering dataframe with highest average hour
highest_average_hours = joined_dataframe.filter(joined_dataframe['seconds'] > average_value)

In [17]:
# getting total number of highest average hours users
highest_average_hours.count()

57

In [20]:
# printing top 20 users with highest average hours
data1 = highest_average_hours.select("user_name")
data1.show()

+--------------------+
|           user_name|
+--------------------+
|ashutoshrit64@gma...|
|giridhardandikwar...|
|salinabodale73@gm...|
|indrajeetgajbhiye...|
|khairnarswapna99@...|
|aman.raj251@gmail...|
|Krushnanikam26@gm...|
|magadum.iranna@gm...|
|puruissimple@gmai...|
|prajktasraut16@gm...|
|samruddhichitnis0...|
|surajwarbhuvan192...|
| grmule018@gmail.com|
|  shelkeva@gmail.com|
|akshaypatwari24@g...|
|rohitsarate1027@g...|
|sharlawar77@gmail...|
|kaleshwetaanil@gm...|
|rahilstar11@gmail...|
|antonyalexcm@gmai...|
+--------------------+
only showing top 20 rows



In [31]:
df = data1
df.write.csv('hdfs://localhost:54310/home/hadoopuser/exmp.csv')

AnalysisException: 'path hdfs://localhost:54310/home/hadoopuser/exmp.csv already exists.;'